In [1]:
import time
import pandas as pd
start_time = time.time()
df=pd.read_csv(r'../input/imstest2/test2.csv', delimiter=',')
print("--- Executed in %s seconds ---" % (time.time() - start_time))

In [2]:
df

In [3]:
!pip install pyfftw

In [4]:
from math import log
import numpy as np
from scipy.stats import entropy
from scipy.signal import hilbert
import pyfftw
from psutil import cpu_count
from scipy.signal import detrend as scipy_detrend
pyfftw.interfaces.cache.enable()
class Features:
    def __init__(self, df):
        self.x = df
    '''
    Time Domain Features
    x = input vib
    p= a sequence of the (discrete) distribution where p[i] is the (possibly unnormalized) probability of event i.
    '''
    def mean(self):
        return np.mean(self.x)
 
    def absoluteMean(self):
        return np.mean(np.abs(self.x))
 
    def standardDeviation(self):
        return np.std(self.x)
 
    def variance(self):
        return np.var(self.x)
 
    def maxAmplitude(self):
        return np.max(self.x)
 
    def minAmplitude(self):
        return np.min(self.x)   
 
    def rms(self):    
        rms = np.sqrt(np.sum(self.x**2)/self.x.size)
        return rms
 
    def peakToPeak(self):
        return np.max(self.x) + np.min(self.x)
 
    def squareMeanRoot(self):
        return np.sum(np.sqrt(np.abs(self.x)))**2

    def standardMoment(self, k):
        xk = (self.x - np.mean(self.x))**k
        x2 = (self.x - np.mean(self.x))**2
        SM = np.mean(xk)/np.mean(x2)**(float(k)/2.0)
        return SM,xk

    def skewness(self):
        return self.x.skew()
 
    def skewnessFactor(self):
        return self.standardMoment(3)[0]/self.rms()**3
 
    def kurtosis(self):
        x2 = np.abs(self.x - np.mean(self.x))**2.0
        K = np.mean(x2**2.0)/self.standardDeviation()**4
        return K
 
    def kurtosisFactor(self):
        return self.standardMoment(4)[0]/self.rms()**4
 
    def clearanceFactor(self):
        return np.max(self.x)/self.squareMeanRoot()
 
    def shapeFactor(self):
        return self.rms()/self.absoluteMean()
 
    def impulseFactor(self):
        return np.max(self.x)/self.absoluteMean()
 
    def crestFactor(self):
        return np.max(self.x)/self.rms()
 
    def sum(self):
        return np.sum(self.x)
 
    def LOG(self):
        return np.exp(np.mean(np.log(np.abs(self.x))))  

    def entropyFactor(self,p):
        return entropy(p, base=2)
    '''
    Frequency Domain Features
    y = signal of FFT 
    f = Frequency of FFT
    Y = Amplitude of FFT / Spectrum Values
    df = Frequency spacing in Hz
    X = Shaft speed in Hz
    D = Pitch diameter
    d = roller diameter
    n = Number of rollers
    theta = Contact angle in degrees
    bearing = 1D array of Bearing characteristic frequencies in orders (i.e. per revolution)
        bearing[0] - Inner race
        bearing[1] - 2x roller spin frequency
        bearing[2] - Cage frequency
        bearing[3] - Outer race frequency
    sf = Sampling frequency
    Detrend = optional string that detrends the signal using scipy.signal.detrend
         'constant' to remove mean value
         'linear' to remove least squares fit
         'none' to do nothing
    hann =  optional for adding a hanning window if true.
    cons = optional parameter to check whether conservative part of the spectrum should be returned:
         True returns sf/2.56 
         False returns sf/2.0 

    '''


    def analytic_signal(self):
        return hilbert(self.x)

    def fft(self, y, sf, detrend='constant', hann=True, cons=True):  
        y = np.array(y)
        n = y.size
        T = n/sf
        # Check if conservative output is desired
        if cons:
            Fmax = sf/2.56
        else:
            Fmax = sf/2.0
        # number of lines
        numL = int(T*Fmax)
        # mean is removed if desired
        if detrend != 'none':
            y = scipy_detrend(y, type=detrend)
        # for hanning window
        if hann is True:
            y = np.hanning(y.size)*y
        # Discrete Fourier Transform
        Y = self.rawfft(y)
        df = 1.0/T
        return np.abs(Y[0:numL])*2.0/n, np.fft.fftfreq(numL,df), df

    def rawfft(self, y):    
        y = np.array(y, copy=True)
        Yobject = pyfftw.builders.fft(y, auto_align_input=True, auto_contiguous=True, planner_effort='FFTW_ESTIMATE', overwrite_input=True)
        return Yobject()   

    def maxPowerSpectrum(self, y):
        fourier= self.rawfft(y)
        abs_ft = np.abs(fourier[0])
        ps= np.square(abs_ft)
        return np.max(ps)    

    def maxEnvelope(self,y):
        y = np.abs(y)
        return np.max(y)
            
    def frequencyCenter(self, Y, f):
        return np.sum(f*Y)/np.sum(Y)

    def rootMeanSquareFrequency(self, Y,f):
        return (np.sum(f**2*Y)/np.sum(Y))**0.5

    def VarianceFrequency(self, Y, f):
        fi=np.mean(f)
        return np.sum(((f-fi)**2)*Y)/np.sum(Y)

    def rootVarianceFrequency(self, f, Y):
        fi=np.mean(f)
        return (np.sum(((f-fi)**2)*Y)/np.sum(Y))**0.5

    def medianFrequency(self, Y, df):
        cumsum = np.cumsum(Y)
        return np.argmin(np.abs(cumsum - 0.5*cumsum[-1]))*df
    
    def bearingcharfrequencies(D, d, n, theta=0.0):
        theta = theta*np.pi/180.0
        FTF = 1.0/2.0 * (1.0 - d/D*np.cos(theta))
        BPFO = n*FTF
        BPFI = n/2.0 * (1.0 + d/D*np.cos(theta))
        BSF = D/(2.0*d) * (1.0 - (d/D * np.cos(theta))**2.0)
       # gives bearing = Bearing fault orders (inner, roller, cage, outer)
        return np.array([BPFI, 2*BSF, FTF, BPFO])

    def bearingEnergy(Y, df, X, bearing):
        lowerFrequency = np.min([bearing[0], bearing[1], bearing[3]])*X*0.95
        upperFrequency = np.max([bearing[0], bearing[1], bearing[3]])*X*1.05
        i1 = int(np.floor(lowerFrequency/df))
        i2 = int(np.ceil(upperFrequency/df))
        return np.sum(Y[i1:i2+1])
    
    

In [5]:
x1=Features(df['bx1'])
d = {}
d['mean'] = x1.mean()
d['absoluteMean'] = x1.absoluteMean()
d['std'] = x1.standardDeviation()
d['var'] = x1.variance()
d['maxA'] = x1.maxAmplitude()
d['minA'] = x1.minAmplitude()
d['rms'] = x1.rms()
d['p2p'] = x1.peakToPeak()
d['skewness'] = x1.skewness()
d['skewnessFactor'] = x1.skewnessFactor()
d['kurtosis'] = x1.kurtosis()
d['kurtosisFactor'] = x1.kurtosisFactor()
d['clearanceFactor'] = x1.clearanceFactor()
d['shapeFactor'] = x1.shapeFactor()
d['crestFactor'] = x1.crestFactor()
d['impulsiveFactor'] = x1.impulseFactor()
d['sum'] = x1.sum()
d['log'] = x1.LOG()
d['entropyF'] = x1.entropyFactor(0.2)
print(d)
fd = {}
fd['signal'] = x1.analytic_signal()
fd['fft'] = x1.fft(fd['signal'], 1024)
fd['maxpowerspectrum'] = x1.maxPowerSpectrum(fd['signal'])
fd['maxEnvelope']=x1.maxEnvelope(fd['signal'])
fd['frequencyCenter']=x1.frequencyCenter(fd['fft'][0],fd['fft'][1])
fd['rootMeanSquareFrequency']=x1.rootMeanSquareFrequency(fd['fft'][0],fd['fft'][1])
fd['varianceFrequency']=x1.VarianceFrequency(fd['fft'][0],fd['fft'][1])
fd['rootVarianceFrequency']=x1.rootVarianceFrequency(fd['fft'][0],fd['fft'][1])
fd['medianFrequency']=x1.medianFrequency(fd['fft'][0],fd['fft'][2])
fd

<a href="./test3.csv"> Download File </a>



In [6]:
fd = {}
fd['signal'] = x1.analytic_signal()
fd['fft'] = x1.fft(fd['signal'], 1024)
fd['maxpowerspectrum'] = x1.maxPowerSpectrum(fd['signal'])
fd['maxEnvelope']=x1.maxEnvelope(fd['signal'])
fd['frequencyCenter']=x1.frequencyCenter(fd['fft'][0],fd['fft'][1])
fd['rootMeanSquareFrequency']=x1.rootMeanSquareFrequency(fd['fft'][0],fd['fft'][1])
fd['varianceFrequency']=x1.VarianceFrequency(fd['fft'][0],fd['fft'][1])
fd['rootVarianceFrequency']=x1.rootVarianceFrequency(fd['fft'][0],fd['fft'][1])
fd['medianFrequency']=x1.medianFrequency(fd['fft'][0],fd['fft'][2])
fd

In [7]:
import matplotlib.pyplot as plt  
f = plt.figure(figsize=(10,10))
plt.subplot(2, 2, 1)
plt.plot(df['bx1'])
plt.title('bx1')
plt.subplot(2, 2, 2)
plt.plot(df['by1'])
plt.title('by1')
plt.subplot(2, 2, 3)
plt.plot(df['bx2'])
plt.title('bx2')
plt.subplot(2, 2, 4)
plt.plot(df['by2'])
plt.title('by2')
plt.show()

In [8]:
y1=Features(df['by1'])
d1 = {}
d1['mean'] = y1.mean()
d1['absoluteMean'] = y1.absoluteMean()
d1['std'] = y1.standardDeviation()
d1['var'] = y1.variance()
d1['maxA'] = y1.maxAmplitude()
d1['minA'] = y1.minAmplitude()
d1['rms'] = y1.rms()
d1['p2p'] = y1.peakToPeak()
d1['skewness'] = y1.skewness()
d1['skewnessFactor'] = y1.skewnessFactor()
d1['kurtosis'] = y1.kurtosis()
d1['kurtosisFactor'] = y1.kurtosisFactor()
d1['clearanceFactor'] = y1.clearanceFactor()
d1['shapeFactor'] = y1.shapeFactor()
d1['crestFactor'] = y1.crestFactor()
d1['impulsiveFactor'] = y1.impulseFactor()
d1['sum'] = y1.sum()
d1['log'] = y1.LOG()
d1['entropyF'] = y1.entropyFactor(0.2)
print(d1)
fd1 = {}
fd1['signal'] = y1.analytic_signal()
fd1['fft'] = y1.fft(fd1['signal'], 1024)
fd1['maxpowerspectrum'] = y1.maxPowerSpectrum(fd1['signal'])
fd['maxEnvelope']=y1.maxEnvelope(fd1['signal'])
fd1['frequencyCenter']=y1.frequencyCenter(fd1['fft'][0],fd1['fft'][1])
fd1['rootMeanSquareFrequency']=y1.rootMeanSquareFrequency(fd1['fft'][0],fd1['fft'][1])
fd1['varianceFrequency']=y1.VarianceFrequency(fd1['fft'][0],fd1['fft'][1])
fd1['rootVarianceFrequency']=y1.rootVarianceFrequency(fd1['fft'][0],fd1['fft'][1])
fd1['medianFrequency']=y1.medianFrequency(fd1['fft'][0],fd1['fft'][2])
fd1

In [9]:
x2=Features(df['bx2'])
d2 = {}
d2['mean'] = x2.mean()
d2['absoluteMean'] = x2.absoluteMean()
d2['std'] = x2.standardDeviation()
d2['var'] = x2.variance()
d2['maxA'] = x2.maxAmplitude()
d2['minA'] = x2.minAmplitude()
d2['rms'] = x2.rms()
d2['p2p'] = x2.peakToPeak()
d2['skewness'] = x2.skewness()
d2['skewnessFactor'] = x2.skewnessFactor()
d2['kurtosis'] = x2.kurtosis()
d2['kurtosisFactor'] = x2.kurtosisFactor()
d2['clearanceFactor'] = x2.clearanceFactor()
d2['shapeFactor'] = x2.shapeFactor()
d2['crestFactor'] = x2.crestFactor()
d2['impulsiveFactor'] = x2.impulseFactor()
d2['sum'] = x2.sum()
d2['log'] = x2.LOG()
d2['entropyF'] = x2.entropyFactor(0.2)
print(d2)
fd2 = {}
fd2['signal'] = x2.analytic_signal()
fd2['fft'] = x2.fft(fd2['signal'], 1024)
fd2['maxpowerspectrum'] = x2.maxPowerSpectrum(fd2['signal'])
fd2['maxEnvelope']=x2.maxEnvelope(fd2['signal'])
fd2['frequencyCenter']=x2.frequencyCenter(fd2['fft'][0],fd2['fft'][1])
fd2['rootMeanSquareFrequency']=x2.rootMeanSquareFrequency(fd2['fft'][0],fd2['fft'][1])
fd2['varianceFrequency']=x2.VarianceFrequency(fd2['fft'][0],fd2['fft'][1])
fd2['rootVarianceFrequency']=x2.rootVarianceFrequency(fd2['fft'][0],fd2['fft'][1])
fd2['medianFrequency']=x2.medianFrequency(fd2['fft'][0],fd2['fft'][2])
fd2

In [10]:
y2=Features(df['by2'])
d3 = {}
d3['mean'] = y2.mean()
d3['absoluteMean'] = y2.absoluteMean()
d3['std'] = y2.standardDeviation()
d3['var'] = y2.variance()
d3['maxA'] = y2.maxAmplitude()
d3['minA'] = y2.minAmplitude()
d3['rms'] = y2.rms()
d3['p2p'] = y2.peakToPeak()
d3['skewness'] = y2.skewness()
d3['skewnessFactor'] = y2.skewnessFactor()
d3['kurtosis'] = y2.kurtosis()
d3['kurtosisFactor'] = y2.kurtosisFactor()
d3['clearanceFactor'] = y2.clearanceFactor()
d3['shapeFactor'] = y2.shapeFactor()
d3['crestFactor'] = y2.crestFactor()
d3['impulsiveFactor'] = y2.impulseFactor()
d3['sum'] = y2.sum()
d3['log'] = y2.LOG()
d3['entropyF'] = y2.entropyFactor(0.2)
print(d3)
fd3 = {}
fd3['signal'] = y2.analytic_signal()
fd3['fft'] = y2.fft(fd3['signal'], 1024)
fd3['maxpowerspectrum'] = y2.maxPowerSpectrum(fd3['signal'])
fd['maxEnvelope']=y2.maxEnvelope(fd3['signal'])
fd3['frequencyCenter']=y2.frequencyCenter(fd3['fft'][0],fd3['fft'][1])
fd3['rootMeanSquareFrequency']=y2.rootMeanSquareFrequency(fd3['fft'][0],fd3['fft'][1])
fd3['varianceFrequency']=y2.VarianceFrequency(fd3['fft'][0],fd3['fft'][1])
fd3['rootVarianceFrequency']=y2.rootVarianceFrequency(fd3['fft'][0],fd3['fft'][1])
fd3['medianFrequency']=y2.medianFrequency(fd3['fft'][0],fd3['fft'][2])
fd3

In [11]:
import matplotlib.pyplot as plt
f = plt.figure(figsize=(12,12))
plt.subplot(2, 2, 1)
plt.plot(fd['fft'][1],fd['fft'][0])
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
plt.title('bx1')
plt.subplot(2, 2, 2)
plt.plot(fd1['fft'][1],fd1['fft'][0])
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
plt.title('by1')
plt.subplot(2, 2, 3)
plt.plot(fd2['fft'][1],fd2['fft'][0])
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
plt.title('bx2')
plt.subplot(2, 2, 4)
plt.plot(fd3['fft'][1],fd3['fft'][0])
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
plt.title('by2')
plt.show()

In [12]:
newdf = pd.DataFrame.from_dict([d])
newdf=newdf.rename(index={0: "bx1"})
newdf

In [13]:
newdf1 = pd.DataFrame.from_dict([d1])
newdf1=newdf1.rename(index={0: "by1"})
newdf1

In [14]:
newdf2 = pd.DataFrame.from_dict([d2])
newdf2=newdf2.rename(index={0: "bx2"})
newdf2

In [15]:
newdf3 = pd.DataFrame.from_dict([d3])
newdf3=newdf3.rename(index={0: "by2"})
newdf3

In [16]:
frames = [newdf, newdf1, newdf2, newdf3]
result = pd.concat(frames)

In [17]:
result

In [18]:
fd.pop('signal')
fd.pop('fft')
dfx1 = pd.DataFrame.from_dict([fd])
dfx1=dfx1.rename(index={0: "bx1"})
dfx1

In [19]:
fd1.pop('signal')
fd1.pop('fft')
dfy1 = pd.DataFrame.from_dict([fd1])
dfy1=dfy1.rename(index={0: "by1"})
dfy1

In [20]:
fd2.pop('signal')
fd2.pop('fft')
dfx2 = pd.DataFrame.from_dict([fd2])
dfx2=dfx2.rename(index={0: "bx2"})
dfx2

In [21]:
fd3.pop('signal')
fd3.pop('fft')
dfy2 = pd.DataFrame.from_dict([fd3])
dfy2=dfy2.rename(index={0: "by2"})

In [22]:
frames1 = [dfx1, dfy1, dfx2, dfy2]
result1 = pd.concat(frames1)

In [23]:
result1

# APPROACH 2, DNE
## To Note: Slide Size and Window Size

In [24]:
from numpy import genfromtxt                #CSV TO NUMPY
from scipy.fftpack import fft			  #FREQ FEATURES	
import matplotlib.pyplot as plt             #PLOT 
import copy                                 #DATA PROCESSING
import numpy as np

start=0
slide=512       #(FOR WINDOWING)WITHIN TIME SAMPLE FRAMING
window=1024    #(FOR SAMPLING)SIZE OF TIME SAMPLE STUDIED AT 2 PT. OF TIME
sample=np.empty((0,window))                #TIME SAMPLE
time_features=np.empty((0,5))   #TIME DOMAIN statistical FEATURES
repeat=int(window/slide)                       #TO SEE HOW MANYWINDOWS IN 1 TIME SAMPLE
hamming=np.hamming(slide)                   #HAMMING WINDOW 
fft_fea=np.empty((0))                     #FREQ DOMAIN  STATISTICAL FEATURES
combined_val=np.empty((0,6))              #FEATURE VECTOR AND LABEL FILE

### MAKING TIME FRAMES/SAMPLES 

In [25]:
import time
start_time = time.time()
count=0
my_data = df
while((start+window) <= (my_data.size)):
    print("Size: (",start,",",start+window,")",count)
    slice1=copy.copy(my_data[start:start+window])
    sample = np.append(sample, [slice1],0 )
    start=start+slide
    count=count+1   
print(sample)
print("--- %s seconds ---" % (time.time() - start_time))

### Last entry of sample

### EXTRACTING TIME DOMAIN FEATURES

In [ ]:
import time
start_time = time.time()
i=0
while(i<count):
    #print(i)
    min_val=np.min(sample[i])
    max_val=np.max(sample[i])
    std_val=np.std(sample[i])
    rms_val= np.sqrt(np.mean(sample[i]**2))
    grad=np.mean(np.gradient(sample[i]))
    temp_array=[min_val,max_val,std_val,rms_val,grad]
    time_features=np.append(time_features, [temp_array],0)
    i=i+1
print(time_features)    
print(len(time_features))
print("--- Executed in %s seconds ---" % (time.time() - start_time))

### EXTRACTING FREQUENCY DOMAIN FEATURES

In [ ]:
#window =1024, slide=512
# 2*512
### import time
freq_features=np.empty((0,int(window/slide),int(slide/2)-1)) #time_Sample,no. of sliding windows in time sample, no.of fft
start_time = time.time()
i=0
fea_counter=0
while(i<count):
    start=0
    freq_sample=np.empty((0,int(slide/2)-1))
    while (start+slide<=window):
        fft_value=fft(sample[i][start:start+slide]*hamming)
        fft_val = 2.0/slide * np.abs(fft_value[0:int(slide/2)-1]) 
        freq_sample = np.append(freq_sample, [fft_val],0)
        for j in range(int(slide/2)-1):
            psd=np.mean(fft_val[j]**2)
        fft_fea=np.append(fft_fea,[psd],0)
        start=start+slide
    freq_features=np.append(freq_features,[freq_sample],0)
    i=i+1
print(fft_fea.shape)
print(freq_features.shape)
print("--- Executed in %s seconds ---" % (time.time() - start_time))
#255 is out of index for size 255

In [ ]:
print(freq_features)

In [ ]:
print(len(freq_features))

### COMBINING BOTH THE FEATURES AND EXTRACTING THEM IN A CSV

In [ ]:
start_time = time.time() 
i=0
fft_counter=0

while(i<time_features.shape[0]):
    var=0
    while(var<repeat):
        join=np.append(time_features[i][:],fft_fea[fft_counter])
#...........for ftt stat value append
#         join=np.concatenate((time_features[i][:],freq_features[i][var][:]))............for fft value append
        combined_val= np.append(combined_val,[join],0)
        fft_counter=fft_counter+1
        var=var+1  
    i=i+1     
# label=np.zeros((combined_val.shape[0],1))
label=np.ones((combined_val.shape[0],1))

combined_val=np.concatenate((combined_val,label),axis=1)
out = open(r'./test1x1.csv', 'w')
for row in combined_val:
    for data_val in row:
        out.write('%f,' % data_val)
    out.write('\n')
out.close()
print("--- Executed in %s seconds ---" % (time.time() - start_time))

### VISUALIZING THE FEATURES IN TIME AND FREQUENCY DOMAIN

In [ ]:
start_time = time.time() 
import matplotlib.gridspec as gridspec
start=0
slide=512
window=1024
f_values = np.arange(0.0, (slide/2)-1)
i=0

while(i<count):
    #TIME DOMAIN GRAPH 
    x = np.arange(start,start+window)
    main_figure = plt.figure(figsize=(20,5))
    main_figure.suptitle(str(i)+ ' Chunk of X1 ',fontsize=20)
    spec = gridspec.GridSpec(2,repeat,main_figure)
    time_fig=plt.subplot(spec[0,:])
#     print(sample[i])
    time_fig.set_ylim([-1.5,2])
    time_fig.plot(x,sample[i])
    time_fig.grid()
    var=0
    #FREQ DOMAIN GRAPH
    while(var<repeat):
        freq_fig=plt.subplot(spec[1,var])
        freq_fig.plot(f_values,freq_features[i][var][:])
        freq_fig.set_ylim([0,0.3])
        freq_fig.grid()
        var=var+1
    start=start+slide
    main_figure.savefig(r'x1test', format="JPEG")
    i=i+1
print("--- Executed in %s seconds ---" % (time.time() - start_time))